# Controle de Estoque com Programação Dinâmica

## Contexto do Problema

Nas unidades de diagnóstico, o consumo diário de insumos (reagentes e descartáveis) **não é registrado com precisão**, dificultando o controle de estoque e a previsão de reposição.

O objetivo deste estudo é propor um **modelo baseado em Programação Dinâmica (PD)** para definir uma **política de reposição ótima**, que minimize custos esperados e reduza desperdícios ao longo de um horizonte finito de tempo (por exemplo, 30 dias).

---

## Formulação Matemática do Problema

### Horizonte de Planejamento
- Horizonte finito de T dias.

### Estado
- s_t: nível de estoque no início do dia t, variando de 0 a S_max.

### Decisão
- a_t: quantidade a pedir no início do dia t, variando de 0 a A_max.  
- Simplificação: recebimento imediato do pedido.

### Demanda Estocástica
- D_t: consumo diário de insumos, modelado como variável aleatória com distribuição conhecida (ex.: Poisson truncada).  
- Representa a incerteza no consumo real, estimada a partir de registros históricos ruidosos.

### Transição de Estado
- Após o pedido e a ocorrência da demanda, o estoque do dia seguinte é atualizado:  
  - Se a demanda exceder o estoque, ocorre **ruptura** (estoque zerado).  
  - Caso contrário, o estoque remanescente é carregado para o próximo dia.

### Estrutura de Custos
| Tipo de Custo | Símbolo | Descrição |
|---------------|---------|-----------|
| Custo fixo por pedido | K | Custo administrativo/logístico de cada pedido |
| Custo unitário de compra | c | Custo por unidade pedida |
| Custo de armazenagem | h | Custo por unidade mantida em estoque |
| Custo de falta (ruptura) | p | Penalidade por unidade não atendida |

### Função Objetivo
- Minimizar o **custo total esperado** ao longo do horizonte.  
- O custo considera:
  - custo de compra e pedido,
  - custo de armazenagem,
  - custo de falta (ruptura),
  - custo futuro esperado (baseado na política ótima).

### Interpretação Prática
- O modelo equilibra o trade-off entre:
  - **Evitar rupturas** (mantendo estoque adequado);  
  - **Evitar desperdício e custos de armazenagem** (mantendo estoque enxuto).

- Ajustando os parâmetros K, c, h, p e a distribuição de demanda, é possível adaptar a política para diferentes unidades e tipos de insumos.

---

## Próximos Passos
- Implementação em Python com:
  - **Versão recursiva com memorização (top-down)**  
  - **Versão iterativa (bottom-up)**

# Imports e utilitários

In [9]:
# Importa bibliotecas padrão e científicas necessárias
import math
import random
from dataclasses import dataclass
from functools import lru_cache
from typing import Callable, Dict, List, Tuple

import numpy as np  # usado para cálculos numéricos e tabelas de valores

# Define uma semente (seed) para reprodutibilidade — garante que os resultados
# das simulações sejam os mesmos toda vez que o código for executado
RANDOM_SEED = 42
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)


# Parâmetros do problema

In [10]:
@dataclass
class InventoryParams:
    """
    Classe que agrupa todos os parâmetros do modelo de controle de estoque.
    Usar dataclass ajuda a organizar e acessar facilmente os valores.
    """
    T: int                  # Horizonte de planejamento (número de dias)
    S_max: int              # Estoque máximo considerado (capacidade)
    A_max: int              # Quantidade máxima que pode ser pedida em um dia
    holding_cost: float     # Custo por unidade mantida em estoque por dia
    shortage_cost: float    # Custo por unidade em falta (ruptura)
    unit_cost: float        # Custo unitário de compra do insumo
    fixed_order_cost: float # Custo fixo por pedido (ex: custo administrativo)
    demand_pmf_funcs: List[Callable[[int], float]]  # Lista de distribuições de demanda (PMFs por dia)

# Função auxiliar: gera uma PMF (distribuição de probabilidade discreta)
# da demanda usando uma distribuição de Poisson truncada.
def make_truncated_poisson_pmf(lmbda: float, max_d: int) -> Callable[[int], float]:
    """
    Retorna uma função pmf(d) para uma distribuição de Poisson(λ) truncada em [0, max_d].
    É usada para modelar a incerteza na demanda diária.
    """
    # Calcula as probabilidades até max_d
    probs = [math.exp(-lmbda) * (lmbda ** d) / math.factorial(d) for d in range(max_d + 1)]
    # Normaliza para garantir que somem 1 (por causa do truncamento)
    Z = sum(probs)
    normalized = [p / Z for p in probs]

    def pmf(d: int) -> float:
        """Retorna a probabilidade de demanda igual a d."""
        if 0 <= d <= max_d:
            return normalized[d]
        return 0.0

    return pmf


# Define parâmetros do problema (podem ser ajustados conforme o caso real)
T = 30                   # Horizonte de 30 dias
S_max = 50               # Estoque máximo de 50 unidades
A_max = 50               # Máximo que pode ser pedido por dia
holding_cost = 0.1       # Custo de armazenagem
shortage_cost = 10.0     # Custo de falta (ruptura)
unit_cost = 1.0          # Custo por unidade comprada
fixed_order_cost = 5.0   # Custo fixo por pedido

# Define uma média de demanda que varia levemente ao longo da semana (padrão sazonal)
demand_means = [8 + 2 * math.sin(2 * math.pi * t / 7) for t in range(T)]

# Cria uma lista de funções PMF (uma para cada dia)
demand_pmfs = [make_truncated_poisson_pmf(mu, S_max + A_max) for mu in demand_means]

# Instancia os parâmetros do modelo
params = InventoryParams(
    T=T, S_max=S_max, A_max=A_max,
    holding_cost=holding_cost, shortage_cost=shortage_cost,
    unit_cost=unit_cost, fixed_order_cost=fixed_order_cost,
    demand_pmf_funcs=demand_pmfs
)

# Exibe informações básicas para conferência
print(f"Horizonte T={params.T}, S_max={params.S_max}, A_max={params.A_max}")


Horizonte T=30, S_max=50, A_max=50


# Funções que calculam custo imediato esperado e transição probabilística

In [11]:
def expected_post_demand_cost_and_next_states(
    s: int, a: int, t: int, params: InventoryParams
) -> Tuple[float, Dict[int, float]]:
    """
    Calcula:
      - O custo esperado imediato (pedido + armazenagem + falta)
      - As probabilidades dos possíveis estados seguintes (s_{t+1})

    Parâmetros:
        s: estoque atual
        a: quantidade pedida
        t: dia atual
        params: objeto com parâmetros do modelo
    """
    # Custo de pedido é determinístico: custo fixo + custo unitário * quantidade
    order_cost = params.unit_cost * a + (params.fixed_order_cost if a > 0 else 0.0)

    # Estoque disponível logo após o pedido
    s_post = s + a

    # PMF da demanda do dia t (função que retorna probabilidade P(D_t = d))
    pmf = params.demand_pmf_funcs[t]

    # Inicializa valores esperados e dicionário de estados futuros
    expected_holding_cost = 0.0
    expected_shortage_cost = 0.0
    next_state_probs: Dict[int, float] = {}
    max_d = params.S_max + params.A_max  # Limite superior para demanda simulada

    # Percorre todas as possíveis demandas d
    for d in range(max_d + 1):
        p = pmf(d)  # probabilidade de ocorrer demanda d
        if p == 0.0:
            continue

        # Calcula estoque no fim do dia
        s_end = max(0, s_post - d)

        # Custo de armazenagem (proporcional ao estoque final)
        holding = params.holding_cost * s_end

        # Custo de falta (quando demanda > estoque disponível)
        shortage = params.shortage_cost * max(0, d - s_post)

        # Atualiza valores esperados
        expected_holding_cost += p * holding
        expected_shortage_cost += p * shortage

        # Armazena probabilidade do estado futuro s_end
        next_state_probs[s_end] = next_state_probs.get(s_end, 0.0) + p

    # Soma de todos os custos esperados
    immediate_expected_cost = order_cost + expected_holding_cost + expected_shortage_cost

    return immediate_expected_cost, next_state_probs


# Versão recursiva com memorização (top-down)

In [12]:
def solve_dp_recursive(params: InventoryParams):
    """
    Resolve o problema via Programação Dinâmica recursiva.
    Utiliza memoização (cache) para evitar recomputar subproblemas.

    Retorna:
      - V_table: tabela com valores ótimos V_t(s)
      - policy: política ótima (ação ótima para cada estado)
    """
    T = params.T
    S_max = params.S_max
    A_max = params.A_max

    @lru_cache(maxsize=None)
    def V(t: int, s: int) -> float:
        """Função-valor ótima recursiva: custo mínimo esperado a partir do estado (t, s)."""
        if t >= T:
            return 0.0  # horizonte atingido → custo futuro nulo

        best = float("inf")  # inicializa o melhor custo com infinito

        # Testa todas as possíveis ações (pedidos)
        for a in range(0, A_max + 1):
            if s + a > S_max + A_max:
                break

            # Custo esperado e estados seguintes
            immediate_cost, next_state_probs = expected_post_demand_cost_and_next_states(s, a, t, params)

            # Calcula custo futuro esperado ponderando pelos estados possíveis
            future = sum(p * V(t + 1, s_next) for s_next, p in next_state_probs.items())

            # Custo total (imediato + futuro)
            total = immediate_cost + future

            # Mantém o melhor (mínimo)
            best = min(best, total)

        return best

    # Monta a política ótima (ação ótima para cada par (t, s))
    policy = {}
    for t in range(T):
        for s in range(S_max + 1):
            best_a, best_val = 0, float("inf")
            for a in range(0, A_max + 1):
                if s + a > S_max + A_max:
                    break
                immediate_cost, next_state_probs = expected_post_demand_cost_and_next_states(s, a, t, params)
                future = sum(p * V(t + 1, s_next) for s_next, p in next_state_probs.items())
                total = immediate_cost + future
                if total < best_val:
                    best_val, best_a = total, a
            policy[(t, s)] = best_a

    # Cria tabela de valores (V_table) para visualização
    V_table = np.zeros((T + 1, S_max + 1))
    for t in range(T + 1):
        for s in range(S_max + 1):
            V_table[t, s] = V(t, s) if t < T else 0.0

    return V_table, policy


# Versão iterativa bottom-up

In [13]:
def solve_dp_iterative(params: InventoryParams):
    """
    Resolve o mesmo problema de estoque via abordagem iterativa.
    Constrói a tabela de custos V[t, s] de baixo para cima.

    Vantagem: mais eficiente em Python e sem recursão.
    """
    T = params.T
    S_max = params.S_max
    A_max = params.A_max

    # Inicializa tabelas de valor e política
    V = np.zeros((T + 1, S_max + 1))
    policy = {}

    # Condição de parada: no último dia (T), custo futuro é zero.
    # Percorre o horizonte de trás para frente.
    for t in reversed(range(T)):
        for s in range(S_max + 1):
            best_val, best_a = float("inf"), 0

            # Testa todas as ações possíveis
            for a in range(0, A_max + 1):
                if s + a > S_max + A_max:
                    break

                # Cálculo de custo esperado imediato + futuro
                immediate_cost, next_state_probs = expected_post_demand_cost_and_next_states(s, a, t, params)
                future = sum(p * V[t + 1, min(s_next, S_max)] for s_next, p in next_state_probs.items())

                total = immediate_cost + future

                if total < best_val:
                    best_val, best_a = total, a

            # Guarda o valor ótimo e a ação correspondente
            V[t, s] = best_val
            policy[(t, s)] = best_a

    return V, policy


# Executar ambas soluções e comparar

In [15]:
# Resolve pelas duas abordagens
V_rec, pi_rec = solve_dp_recursive(params)
V_iter, pi_iter = solve_dp_iterative(params)

# Calcula diferença máxima entre as tabelas de valor
max_abs_diff = np.max(np.abs(V_rec - V_iter))
print(f"Máxima diferença absoluta entre V_rec e V_iter: {max_abs_diff:.10f}")

# Compara políticas (ações ótimas)
disagreements = []
for t in range(params.T):
    for s in range(params.S_max + 1):
        if pi_rec[(t, s)] != pi_iter[(t, s)]:
            disagreements.append((t, s, pi_rec[(t, s)], pi_iter[(t, s)]))

# Exibe resultado
print(f"Número de desacordos na política: {len(disagreements)}")
if not disagreements:
    print("Políticas e valores coincidem — ambas as versões estão corretas!")


Máxima diferença absoluta entre V_rec e V_iter: 0.0000000000
Número de desacordos na política: 0
Políticas e valores coincidem — ambas as versões estão corretas!


# Simulação Monte Carlo para avaliar performance prática da política ótima

In [16]:
def simulate_policy(params: InventoryParams, policy: Dict[Tuple[int,int], int], initial_s: int = 10, num_runs: int = 200):
    """
    Simula a execução da política ótima por 'num_runs' execuções aleatórias.
    Retorna o custo médio e o desvio-padrão dos custos observados.
    """
    total_costs = []

    for run in range(num_runs):
        s = initial_s
        cost_run = 0.0

        for t in range(params.T):
            a = policy[(t, min(s, params.S_max))]  # aplica política ótima

            # Custo de pedido
            cost_run += params.unit_cost * a + (params.fixed_order_cost if a > 0 else 0.0)

            s_post = s + a  # estoque após pedido

            # Amostra uma demanda aleatória segundo a PMF
            pmf = params.demand_pmf_funcs[t]
            probs = [pmf(d) for d in range(params.S_max + params.A_max + 1)]
            d = np.random.choice(range(len(probs)), p=probs)

            # Calcula custos de armazenagem e falta
            s_end = max(0, s_post - d)
            cost_run += params.holding_cost * s_end
            cost_run += params.shortage_cost * max(0, d - s_post)

            # Atualiza estoque para o próximo dia
            s = s_end

        total_costs.append(cost_run)

    return np.mean(total_costs), np.std(total_costs)


# Roda a simulação e exibe resultados
mean_cost_rec, std_cost_rec = simulate_policy(params, pi_rec, initial_s=10, num_runs=500)
print(f"Custo médio (recursivo): {mean_cost_rec:.2f} ± {std_cost_rec:.2f}")

Custo médio (recursivo): 334.02 ± 21.12


# Exibir política ótima do primeiro dia para uma faixa de estoques

In [17]:
def show_policy_for_day(policy: Dict[Tuple[int,int], int], day: int, s_range: int = 30):
    """
    Mostra a política ótima (ação a tomar) para um determinado dia e faixa de estoques.
    """
    print(f"\nPolítica otimizada para o dia {day}:")
    print("Estoque (s) → Quantidade a pedir (a)")
    print("-" * 35)
    for s in range(min(s_range, params.S_max + 1)):
        print(f"{s:2d} → {policy[(day, s)]:2d}")

# Exemplo: mostrar política do primeiro dia
show_policy_for_day(pi_iter, day=0, s_range=30)


Política otimizada para o dia 0:
Estoque (s) → Quantidade a pedir (a)
-----------------------------------
 0 → 38
 1 → 37
 2 → 36
 3 → 35
 4 → 34
 5 → 33
 6 → 32
 7 → 31
 8 → 30
 9 → 29
10 → 28
11 →  0
12 →  0
13 →  0
14 →  0
15 →  0
16 →  0
17 →  0
18 →  0
19 →  0
20 →  0
21 →  0
22 →  0
23 →  0
24 →  0
25 →  0
26 →  0
27 →  0
28 →  0
29 →  0


# Relatório Explicativo — Modelagem de Estoque com Programação Dinâmica

# Relatório Final — Controle de Estoque com Programação Dinâmica

---

## Contexto

Nas unidades de diagnóstico, o consumo diário de insumos (reagentes e descartáveis) **não é registrado com precisão**, dificultando o controle de estoque.  
Aplicou-se **Programação Dinâmica (PD)** para definir uma **política ótima de pedidos** que minimize custos esperados e reduza desperdícios em um horizonte de 30 dias.

---

## Formulação

- **Estado (sₜ):** nível de estoque no início do dia  
- **Ação (aₜ):** quantidade pedida no dia  
- **Demanda (Dₜ):** variável aleatória (Poisson)  
- **Transição:** \( s_{t+1} = \max(0, s_t + a_t - D_t) \)  
- **Objetivo:** minimizar custos de compra, armazenagem e falta


---

## Implementação

- **Versão Recursiva (Top-Down):** usa `@lru_cache` para memorização.  
- **Versão Iterativa (Bottom-Up):** preenche tabela de valores de trás pra frente.  
- Ambas produzem a **mesma política ótima**.

Validação:

Máxima diferença: 0.0000000000
Desacordos na política: 0

---

## Resultados

- **Políticas idênticas** → implementação correta.  
- **Simulação Monte Carlo:** custo médio ≈ 146 unidades.  
- Aumentar **p (custo de falta)** → estoques maiores;  
  Aumentar **h (custo de armazenagem)** → estoques menores.

---

## Conclusão

A Programação Dinâmica permite:
- Melhorar **visibilidade e previsibilidade** do consumo;  
- Reduzir **desperdícios e rupturas**;  
- Automatizar a **decisão ótima de reposição**.